In [1]:
#Preamble stuff, loading up libraries and convenience functions
import all_data_handler
import pandas
import statsmodels
import statsmodels.api as sm
from statsmodels.formula.api import ols
import itertools
import seaborn as sb
import scipy.stats as ss #For ss.f_oneway() ANOVAs
import re
import matplotlib.pyplot as plt
import numpy as np
import copy

adh = all_data_handler.UserData()

/usr/local/lib/python2.7/dist-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools
/usr/lib/python2.7/dist-packages/matplotlib/__init__.py:874: UserWarning: axes.color_cycle is deprecated and replaced with axes.prop_cycle; please use the latter.
  warnings.warn(self.msg_depr % (key, alt_key))


How many people used voice commands?

Did the text of the instruction slide make the users in the formation task use voice commands more than the other tasks?

Did the stop task provoke voice commands from people who didn't normally use voice commands?

In [2]:
#Get per-task count of voice commands used
def count_voice(participant):
    #Dict of task id to count of taps on robots
    counts = {}
    for task_id in participant["tasks"].keys():
        #Default to no counts of select taps
        counts[task_id] = 0
        events = participant["tasks"][task_id]
        #Flip through all the events
        for event in events:
            if event["event_type"] == "memo":
                #This event is a memo, not a user gesture
                continue
            elif "example" in event.keys() and event["example"] == True:
                #This event is an example, don't count it
                continue
            else:
                #If it's a voice command, count it
                if event["event_type"] == "voice_command":
                    counts[task_id] += 1
    return counts

#Get the per-user, per task counts
voice_counts = adh.apply(count_voice)

#Totals, indexed by task NAME and user id
task_totals = {}
user_totals = {}

for user in voice_counts.keys():
    for task in voice_counts[user]:
        #Check/add for the totals
        if user in user_totals.keys():
            user_totals[user] += voice_counts[user][task]
        else:
            user_totals[user] = voice_counts[user][task]
        
        #Index by name, since the numbers don't match across conditions
        task_name = adh.taskNumberToName(int(task), user)
        if task_name in task_totals.keys():
            task_totals[task_name] += voice_counts[user][task]
        else:
            task_totals[task_name] = voice_counts[user][task]
            
print user_totals
print task_totals

{u'42': 1, u'22': 1, u'43': 0, u'23': 1, u'24': 0, u'25': 6, u'26': 0, u'27': 0, u'20': 0, u'21': 0, u'48': 0, u'49': 0, u'46': 0, u'47': 0, u'44': 0, u'45': 0, u'28': 0, u'29': 0, u'40': 0, u'41': 0, u'1': 0, u'3': 0, u'2': 0, u'5': 0, u'4': 0, u'7': 1, u'6': 0, u'9': 19, u'8': 0, u'39': 0, u'38': 0, u'11': 2, u'10': 1, u'13': 0, u'12': 0, u'15': 2, u'14': 0, u'17': 0, u'16': 0, u'19': 3, u'18': 0, u'31': 0, u'30': 0, u'37': 0, u'36': 0, u'35': 0, u'34': 0, u'33': 0, u'32': 0, u'50': 0}
{'crate_dispersed': 3, 'patrol_a': 2, 'square': 4, 'divide': 1, 'divide_color_1': 1, 'divide_color_mix': 3, 'stop': 3, 'crate': 2, 'remove': 1, 'mark': 1, 'patrol_screen': 1, 'move_a': 2, 'divide_color_2': 1, 'disperse': 2, 'merge': 2, 'line': 5, 'move_wall': 1, 'split': 2}


In [3]:
#What percentage of users used a voice command?
user_count = len(user_totals.keys())
used_voice = {u:c for u, c in user_totals.items() if c > 0}
voice_count = len(used_voice.keys())
percent = 100 * (float(voice_count)/float(user_count))
print percent


20.0


In [4]:
print used_voice

{u'11': 2, u'25': 6, u'15': 2, u'22': 1, u'23': 1, u'19': 3, u'42': 1, u'7': 1, u'9': 19, u'10': 1}


In [5]:
#Try loading all of the survey data 
import json
with open("survey.json", 'r') as survey:
    survey_data = json.loads(survey.read())

In [6]:
survey_data["participants"]['11']["Q17_android_phone"]

u'moderate'

In [7]:
#Phone questions
phone_q = ["Q17_android_phone", "Q17_iphone", "Q17_win_phone", "Q17_palm_pda_phone"]
for p in used_voice.keys():
    print "User {}".format(p)
    for q in phone_q:
        use = survey_data["participants"][p][q]
        print "{} : {}".format(q, use)
        
    

User 11
Q17_android_phone : moderate
Q17_iphone : familiar
Q17_win_phone : complete_unfamiliar
Q17_palm_pda_phone : rarely
User 25
Q17_android_phone : complete_unfamiliar
Q17_iphone : familiar
Q17_win_phone : complete_unfamiliar
Q17_palm_pda_phone : complete_unfamiliar
User 15
Q17_android_phone : familiar
Q17_iphone : familiar
Q17_win_phone : rarely
Q17_palm_pda_phone : complete_unfamiliar
User 22
Q17_android_phone : frequent
Q17_iphone : rarely
Q17_win_phone : moderate
Q17_palm_pda_phone : complete_unfamiliar
User 23
Q17_android_phone : rarely
Q17_iphone : familiar
Q17_win_phone : rarely
Q17_palm_pda_phone : rarely
User 19
Q17_android_phone : moderate
Q17_iphone : familiar
Q17_win_phone : complete_unfamiliar
Q17_palm_pda_phone : complete_unfamiliar
User 42
Q17_android_phone : familiar
Q17_iphone : complete_unfamiliar
Q17_win_phone : complete_unfamiliar
Q17_palm_pda_phone : frequent
User 7
Q17_android_phone : familiar
Q17_iphone : moderate
Q17_win_phone : complete_unfamiliar
Q17_palm_p

How many people used 2-handed gestures, and which gestures were they?

Note that checking the hands flag only gets gestures where one gesture is divided across two hands, mostly pinch. Cases where the user made different gestures with each hand at the same time are coded by having the same end time, so they are detected by building a hash of events keyed by end time and then discarding all the entries that have only one event with that end time.

In [8]:
#Get per-task count of two-handed commands
def get_hands(participant):
    #Dict of task id to commands
    counts = {}
    for task_id in participant["tasks"].keys():
        #Default to no counts commands
        counts[task_id] = []
        events = participant["tasks"][task_id]
        #Flip through all the events
        for event in events:
            if event["event_type"] == "memo":
                #This event is a memo, not a user gesture
                continue
            elif "example" in event.keys() and event["example"] == True:
                #This event is an example, don't count it
                continue
            else:
                #If they used two hands, count it
                if "hands" in event.keys() and event["hands"] > 1:
                    counts[task_id].append(event)
    return counts

two_hand_gestures = adh.apply(get_hands)

def simultaneous_gestures(participant):
    #Dict of task id to commands
    by_time = {}
    for task_id in participant["tasks"].keys():
        #Build per-task dicts
        by_time[task_id] = {}
        events = participant["tasks"][task_id]
        #Flip through all the events
        for event in events:
            #Skip memos and examples
            if event["event_type"] =="memo" or "example" in event.keys() and event["example"] == True:
                continue
            else:
                #Put the event in the hash indexed by what time it occured
                if event["time"] in by_time[task_id].keys():
                    by_time[task_id][event["time"]].append(event)
                else:
                    by_time[task_id][event["time"]] = [event]
        #Now strip out any events that don't have another event at the same time
        #I could probably make this a dict comprehension
        for e_time in by_time[task_id].keys():
            if len(by_time[task_id][e_time]) < 2:
                del by_time[task_id][e_time]
    return by_time
            
same_time = adh.apply(simultaneous_gestures)

In [9]:
#Count how many users made two-handed gestures
two_hand_users = set()
for user in same_time.keys():
    for task in same_time[user].keys():
        #If we have a non-empty dict for any task, add this user to the two-hand users
        #The same check can work for same_time and two_hand_gestures because empty dicts and empty lists 
        #are both falsey, and both dicts have the same key structure
        if same_time[user][task] or two_hand_gestures[user][task]:
            two_hand_users.add(user)
        #If we've already added this user, move on to the next one
        if user in two_hand_users:
            continue

two_hand_users

{u'1',
 u'11',
 u'15',
 u'18',
 u'19',
 u'2',
 u'21',
 u'23',
 u'26',
 u'28',
 u'3',
 u'30',
 u'33',
 u'34',
 u'38',
 u'39',
 u'42',
 u'43',
 u'44',
 u'46',
 u'47',
 u'49',
 u'50',
 u'6',
 u'7'}

In [10]:
#What were the gesture pairs used on each hand?
gesture_pairs = {}
#Simultaneous gestures, for each user
for user in same_time.keys():
    #for each task for that user
    for task in same_time[user].keys():
        #If they had a simultaneous gesture
        if same_time[user][task]:
            for time in same_time[user][task].keys():
                #Build a name for the sequence
                events = ""
                for event in same_time[user][task][time]:
                    events += event["event_type"]
                    events += " "
                events = events.strip()
                #Add to the counts
                if events in gesture_pairs.keys():
                    gesture_pairs[events] += 1
                else:
                    gesture_pairs[events] = 1

for user in two_hand_gestures.keys():
    for task in two_hand_gestures[user].keys():
        if two_hand_gestures[user][task]:
            #This is a list of all the two-hand gestures in this task
            for event in two_hand_gestures[user][task]:
                #Each event is bimanual for the same task type, so just put the type as the key
                events = ""
                if event["event_type"] == "drag":
                    events = "drag_s drag_s"
                elif event["event_type"] == "pinch": 
                    if event["reverse"]:
                        events = "reverse_pinch reverse_pinch"
                    else:
                        events = "pinch pinch"
                else:
                    #This is something I didn't account for, shouldn't print anything
                    print event["event_type"]
                    
                #Add to the counts 
                if events in gesture_pairs.keys():
                    gesture_pairs[events] += 1
                else:
                    gesture_pairs[events] = 1                
                
gesture_pairs
         


{u'box_select box_select': 3,
 u'box_select tap': 1,
 u'box_select ui': 1,
 u'drag drag': 26,
 u'drag tap': 4,
 'drag_s drag_s': 5,
 u'other other': 5,
 'pinch pinch': 22,
 'reverse_pinch reverse_pinch': 7,
 u'tap drag': 2,
 u'tap tap': 6}

In [11]:
#Figure out which tasks got two-handed interactions
task_two_hands = {}
for user in two_hand_gestures.keys():
    for task in two_hand_gestures[user].keys():
        if two_hand_gestures[user][task]:
            #Get the task name 
            task_name = adh.taskNumberToName(int(task), user)
            #It's a list of two-handed gestures, so add all of them
            if task_name in task_two_hands.keys():
                task_two_hands[task_name] += len(two_hand_gestures[user][task])
            else:
                task_two_hands[task_name] = len(two_hand_gestures[user][task])

for user in same_time.keys():
    for task in same_time[user].keys():
        if same_time[user][task]:
            #Get the task name 
            task_name = adh.taskNumberToName(int(task), user)
            #It's a dict of times to gesture pairs, so add all of them
            if task_name in task_two_hands.keys():
                task_two_hands[task_name] += len(same_time[user][task].keys())
            else:
                task_two_hands[task_name] = len(same_time[user][task].keys())
                
task_two_hands
            

{'crate': 1,
 'disperse': 1,
 'divide': 12,
 'divide_color_1': 5,
 'divide_color_2': 9,
 'divide_color_mix': 4,
 'line': 2,
 'merge': 14,
 'move_a': 2,
 'move_wall': 4,
 'patrol_screen': 9,
 'split': 6,
 'square': 12,
 'stop': 1}

In [12]:
#Figure out if use of two-handed gestures is related to condition

In [13]:
#Did any participants have no familiarity with smartphones
familiar = ["familiar", "frequent", "moderate"]
phone_q = ["Q17_android_phone", "Q17_iphone"]#, "Q17_win_phone", "Q17_palm_pda_phone"]

is_familiar = set()
not_familiar = set()
#For all participants
for p in range(1,51):
    for q in phone_q:
        use = survey_data["participants"][str(p)][q]
        #If they're familiar with any phone, we're done here
        if use in familiar:
            is_familiar.add(p)
            break
len(is_familiar)

49

In [14]:
#Print all the games people listed
# for user in survey_data["participants"].keys():
#     games = survey_data["participants"][user]["Q11"]
#     if games != "":
#         print user, games

#Figure out if playing RTS games had an influence on box selection use
#This list of users who play RTS games was compiled by hand, since users listed game titles

rts_players = [2, 19, 26, 27, 3, 30, 38, 50]

#Given the object list of a gesture from the coding, return a list of letters representing normalized 
#values for the objects of the gesture
def tag_object(original):
    #This use of strip is to prevent quotes from messing up regex matches
    original = " ".join(original).strip("\"")
    tags = []
    #Matches robots, robot, bot, bots, etc. 
    robots = re.compile("bot|group|swarm|orange|red", re.I)
    crate = re.compile("crate|wood", re.I)
    targetA = re.compile("area a|box a| a$|to a,|^a$", re.I)
    targetB = re.compile("area b|box b| b$|to b,|^b$", re.I)
    whitespace = re.compile("whitespace|ground|screen|white area", re.I)
    
    toCheck = [(robots, "r"), (crate, "c"), (targetA, "a"), (targetB, "b"), (whitespace, "w")]
    for compiled, tag in toCheck:
        if re.search(compiled, original):
            tags.append(tag)
    return tags

def count_box_select(participant):
    #Dict of task id to count of box selects on robots
    counts = {}
    for task_id in participant["tasks"].keys():
        #Default to no counts
        counts[task_id] = 0
        events = participant["tasks"][task_id]
        #Flip through all the events
        for event in events:
            if event["event_type"] == "memo" or ("example" in event.keys() and event["example"] == True):
                #This event is an example or memo, don't count it
                continue
            elif event["event_type"] == "box_select":
                tags = tag_object(event["objects"])
                #If there's an r in tags, this was a box select on a robot
                if 'r' in tags:
                    counts[task_id] += 1
    return counts

box_select_by_users = adh.apply(count_box_select)

#Now break it up into box select by RTS players and box select by non-RTS players
non_users = []
rts_users = []
for user in box_select_by_users.keys():
    if int(user) in rts_players:
        for task in box_select_by_users[user].keys():
            rts_users.append(box_select_by_users[user][task])
    else:
        for task in box_select_by_users[user].keys():
            non_users.append(box_select_by_users[user][task])

f, p = ss.f_oneway(non_users, rts_users)
non_mean = np.mean(non_users)
non_std = np.std(non_users)
rts_mean = np.mean(rts_users)
rts_std = np.std(rts_users)

print "F:{} p:{}".format(f,p)
print "Non-RTS users total: {} mean: {} std. dev.: {}".format(sum(non_users), non_mean, non_std)
print "RTS users total: {} mean: {} std. dev.: {}".format(sum(rts_users), rts_mean, rts_std)


F:53.8527435084 p:5.26913883899e-13
Non-RTS users total: 51 mean: 0.074780058651 std. dev.: 0.349247656016
RTS users total: 49 mean: 0.3828125 std. dev.: 0.740788154497


In [15]:
#Do RTS gamers want a UI?

def count_ui(participant):
    #Dict of task id to count of box selects on robots
    counts = {}
    for task_id in participant["tasks"].keys():
        #Default to no counts
        counts[task_id] = 0
        events = participant["tasks"][task_id]
        #Flip through all the events
        for event in events:
            if event["event_type"] == "memo" or ("example" in event.keys() and event["example"] == True):
                #This event is an example or memo, don't count it
                continue
            elif event["event_type"] == "ui":
                counts[task_id] += 1
    return counts

ui_uses = adh.apply(count_ui)

#Now break it up by RTS players or not
non_users = []
rts_users = []
for user in ui_uses.keys():
    if int(user) in rts_players:
        for task in ui_uses[user].keys():
            rts_users.append(ui_uses[user][task])
    else:
        for task in ui_uses[user].keys():
            non_users.append(ui_uses[user][task])
            
f, p = ss.f_oneway(non_users, rts_users)
non_mean = np.mean(non_users)
non_std = np.std(non_users)
rts_mean = np.mean(rts_users)
rts_std = np.std(rts_users)

print "F:{} p:{}".format(f,p)
print "Non-RTS users total: {} mean: {} std. dev.: {}".format(sum(non_users), non_mean, non_std)
print "RTS users total: {} mean: {} std. dev.: {}".format(sum(rts_users), rts_mean, rts_std)

F:56.2047911478 p:1.71863428681e-13
Non-RTS users total: 20 mean: 0.0293255131965 std. dev.: 0.18528526685
RTS users total: 74 mean: 0.578125 std. dev.: 1.8608187672


In [16]:
#Determine if Mac OS use had any influence on use of 5f scatter as a gesture for disperse
for user in adh.data.keys():
    tasks = sorted([int(k) for k in adh.data[user]["tasks"].keys()])
    #Only users in the unknown, 10, 100, 1000 conditions had a disperse task
    if tasks[-1] == 16 or tasks[-1] == 18:
        #Get the events for the last task
        events = adh.data[user]["tasks"][str(tasks[-1])]
        for event in events:
            if event["event_type"] == "memo" or ("example" in event.keys() and event["example"] == True):
                #This event is an example or memo, don't count it
                continue
            elif event["event_type"] =="other":
                print user, event["event_type"], " ".join(event["description"])
            elif event["event_type"] == "voice_command":
                print user, event["event_type"], " ".join(event["command"])
            else:
                objects = ""
                if "objects" in event.keys():
                    objects = " ".join(event["objects"])
                print user, event["event_type"], objects

48 drag "perimeter of whitespace"
43 drag "squiggle over all robots"
49 box_select "robots"
49 other "4f scatter"
49 other "4f scatter"
24 lasso "bl quarter of robots"
24 lasso "tl quarter of robots"
24 lasso "tr quarter of robots"
24 lasso "br quarter of robots"
24 drag "bl group to bl corner"
24 drag "center to tl edge of swarm"
24 drag "tr group to outside of group towards tr"
24 drag "bl group to bl corner"
25 voice_command "disperse into the open area"
25 drag "open area/around robots"
26 other "5f scatter"
20 drag "robots to bl corner"
20 drag "robots to bottom"
20 drag "robots to br corner"
20 drag "robots to r side"
20 drag "robots to tr corner"
20 drag "robots to top"
20 drag "robots to tl corner"
20 drag "robots to l"
21 lasso "robots (anticlockwise)"
23 drag "vertically down screen"
23 drag "vertically down screen"
23 drag "vertically down screen"
23 drag "horizontally across screen"
23 drag "horizontally across screen"
23 tap "robot"
23 tap "robot"
23 tap "robot"
23 tap "ro

In [17]:
scatters = [49, 26, 46, 45, 41, 4, 6, 8, 11, 13, 19, 18]
devices = ["Q17_iphone", "Q17_ipad"]
#Determine if the scatter-users were also Mac iOS device users (don't have OS familiarity in data)
for p in scatters:
    use = []
    for d in devices:
        use.append(survey_data["participants"][str(p)][d])
    print p, use, survey_data["participants"][str(p)]["Q7"]

49 [u'familiar', u'frequent'] GIMP
SolidWorks
Meshmixer
Blender
Arduino
26 [u'rarely', u'frequent'] Games like 2048, EverNote
46 [u'moderate', u'moderate'] Web surfing (chrome, etc). Eclipse, Git, Atom.io, Steam, Counter-Strike, Snapchat, facebook, camera.
45 [u'familiar', u'familiar'] word, excel, google chrome, notepad, paint, e-mail, powerpoint
41 [u'rarely', u'rarely'] SolidWorks
Android operating system
Windows 10
Gmail for Android
Any.do
Google Keep
myHomework
Google Calendar
4 [u'familiar', u'familiar'] Microsoft Word/Excel/Powerpoint
Image Lab 
Photoshop/Procreate
Snapchat
Facebook
6 [u'rarely', u'complete_unfamiliar'] Safarai, Chrome, PyCharm, Terminal, an Email client, PDF Reader
8 [u'moderate', u'moderate'] Bash Shell, windows OS, Mac OS, Android Device, Linux
11 [u'familiar', u'moderate'] Office, Empowers, MassLynx, SedFit
13 [u'familiar', u'rarely'] For my phone, I use Flipboard and Facebook on a regular basis.  On my computer I often times use Google Chrome, Excel, and Wo

In [18]:
#What about just gamers vs nongamers for box select, lasso select, and UI gestures?
gamers = []
for p in range(1, 51):
    if survey_data["participants"][str(p)]["Q9"] =="Yes":
        gamers.append(str(p))

ui_uses = adh.apply(count_ui)

#Now break it up by RTS players or not
gamer_users = []
non_gamer_users = []
for user in ui_uses.keys():
    if user in gamers:
        for task in ui_uses[user].keys():
            gamer_users.append(ui_uses[user][task])
    else:
        for task in ui_uses[user].keys():
            non_gamer_users.append(ui_uses[user][task])
            
f, p = ss.f_oneway(non_gamer_users, gamer_users)
non_mean = np.mean(non_gamer_users)
non_std = np.std(non_gamer_users)
game_mean = np.mean(gamer_users)
game_std = np.std(gamer_users)

print "F:{} p:{}".format(f,p)
print "Non-gamer users total: {} mean: {} std. dev.: {}".format(sum(non_gamer_users), non_mean, non_std)
print "Gamer users total: {} mean: {} std. dev.: {}".format(sum(gamer_users), game_mean, game_std)

F:5.45016625178 p:0.0198105893262
Non-gamer users total: 4 mean: 0.0167364016736 std. dev.: 0.12828208968
Gamer users total: 90 mean: 0.15761821366 std. dev.: 0.92807387755


In [19]:
#Box select, looking at just gamers rather than RTS players
box_select_by_users = adh.apply(count_box_select)

#Now break it up into box select by RTS players and box select by non-RTS players
non_users = []
rts_users = []
for user in box_select_by_users.keys():
    if user in gamers:
        for task in box_select_by_users[user].keys():
            rts_users.append(box_select_by_users[user][task])
    else:
        for task in box_select_by_users[user].keys():
            non_users.append(box_select_by_users[user][task])

f, p = ss.f_oneway(non_users, rts_users)
non_mean = np.mean(non_users)
non_std = np.std(non_users)
rts_mean = np.mean(rts_users)
rts_std = np.std(rts_users)

print "F:{} p:{}".format(f,p)
print "Non-gamers total: {} mean: {} std. dev.: {}".format(sum(non_users), non_mean, non_std)
print "Gamers total: {} mean: {} std. dev.: {}".format(sum(rts_users), rts_mean, rts_std)


F:11.3061273283 p:0.000808848311725
Non-gamers total: 10 mean: 0.0418410041841 std. dev.: 0.220132994778
Gamers total: 90 mean: 0.15761821366 std. dev.: 0.512222520279


In [20]:
#Lasso select, looking at just gamers rather than RTS players
def count_lasso_select(participant):
    #Dict of task id to count of box selects on robots
    counts = {}
    for task_id in participant["tasks"].keys():
        #Default to no counts
        counts[task_id] = 0
        events = participant["tasks"][task_id]
        #Flip through all the events
        for event in events:
            if event["event_type"] == "memo" or ("example" in event.keys() and event["example"] == True):
                #This event is an example or memo, don't count it
                continue
            elif event["event_type"] == "lasso":
                tags = tag_object(event["objects"])
                #If there's an r in tags, this was a box select on a robot
                if 'r' in tags:
                    counts[task_id] += 1
    return counts

lasso_select_by_users = adh.apply(count_lasso_select)

#Now break it up into box select by gamers and non-gamers
non_users = []
gamer_users = []
for user in lasso_select_by_users.keys():
    if user in gamers:
        for task in lasso_select_by_users[user].keys():
            gamer_users.append(lasso_select_by_users[user][task])
    else:
        for task in lasso_select_by_users[user].keys():
            non_users.append(lasso_select_by_users[user][task])

f, p = ss.f_oneway(non_users, rts_users)
non_mean = np.mean(non_users)
non_std = np.std(non_users)
gamer_mean = np.mean(gamer_users)
gamer_std = np.std(gamer_users)

print "F:{} p:{}".format(f,p)
print "Non-gamers total: {} mean: {} std. dev.: {}".format(sum(non_users), non_mean, non_std)
print "Gamers total: {} mean: {} std. dev.: {}".format(sum(gamer_users), gamer_mean, gamer_std)

F:0.38854276741 p:0.533242287923
Non-gamers total: 44 mean: 0.18410041841 std. dev.: 0.633445719423
Gamers total: 104 mean: 0.182136602452 std. dev.: 0.553697637849


In [21]:
#Mark saw a lack of pinch gestures in RTS gamers vs. everyone else
#Check if that's a thing
def count_pinch(participant):
    #Dict of task id to count of box selects on robots
    counts = {}
    for task_id in participant["tasks"].keys():
        #Default to no counts
        counts[task_id] = 0
        events = participant["tasks"][task_id]
        #Flip through all the events
        for event in events:
            if event["event_type"] == "memo" or ("example" in event.keys() and event["example"] == True):
                #This event is an example or memo, don't count it
                continue
            elif event["event_type"] == "pinch":
                counts[task_id] += 1
    return counts

pinches = adh.apply(count_pinch)

#Redefining here just in case it got scribbled over
rts_players = [2, 19, 26, 27, 3, 30, 38, 50]

#Now break it up into box select by RTS gamers and everyone else
non_users = []
gamer_users = []
for user in pinches.keys():
    if user in rts_players:
        for task in pinches[user].keys():
            gamer_users.append(pinches[user][task])
    else:
        for task in pinches[user].keys():
            non_users.append(pinches[user][task])

f, p = ss.f_oneway(non_users, rts_users)
            
non_mean = np.mean(non_users)
non_std = np.std(non_users)
gamer_mean = np.mean(gamer_users)
gamer_std = np.std(gamer_users)

print "F:{} p:{}".format(f,p)
print "Non-gamers total: {} mean: {} std. dev.: {}".format(sum(non_users), non_mean, non_std)
print "Gamers total: {} mean: {} std. dev.: {}".format(sum(gamer_users), gamer_mean, gamer_std)

F:14.5904918373 p:0.000139522042845
Non-gamers total: 51 mean: 0.062962962963 std. dev.: 0.406412927654
Gamers total: 0 mean: nan std. dev.: nan


/usr/local/lib/python2.7/dist-packages/numpy/core/fromnumeric.py:2957: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/usr/local/lib/python2.7/dist-packages/numpy/core/_methods.py:80: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/usr/local/lib/python2.7/dist-packages/numpy/core/_methods.py:135: RuntimeWarning: Degrees of freedom <= 0 for slice
  keepdims=keepdims)
/usr/local/lib/python2.7/dist-packages/numpy/core/_methods.py:105: RuntimeWarning: invalid value encountered in true_divide
  arrmean, rcount, out=arrmean, casting='unsafe', subok=False)
/usr/local/lib/python2.7/dist-packages/numpy/core/_methods.py:127: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [22]:
#Print all the memos to look for ambiguities that were mentioned in coding
def print_memos(participant):
    #Dict of task id to count of box selects on robots
    counts = {}
    for task_id in participant["tasks"].keys():
        #Default to no counts
        counts[task_id] = 0
        events = participant["tasks"][task_id]
        #Flip through all the events
        for event in events:
            if event["event_type"] == "memo":
                #get the task name
                task = adh.taskNumberToName(number=int(task_id), pId=str(participant["participant"]))
                print "{} {} {}".format(participant["participant"], task, " ".join(event["description"]))

adh.apply(print_memos)

42 patrol_screen "alternate method was marking points for patrol"
42 stop "doubleclick restarts stopped robot after hold"
42 divide_color_2 "user described robots removing obstacles"
42 crate "mentioned having voice control to tell the robot to pick up the box"
48 square "mentioned using duckling approach again"
48 square "defning square by points rather than line"
48 square "mentioned having a panel on the side for formation control"
48 line "said robots could sense each other and follow-the-leader 'like ducklings'"
48 line "examples about how tedious controlling each robot to postion would be"
48 line "examples about numbering the robots"
48 remove "talks about team creation with side menu"
48 disperse "disperse away from a point"
48 disperse "disperse randomly, mentions having a random wander function, no contact"
48 disperse "using formations and space, define an area"
48 disperse "says 'hit fill volume or something', no contact"
48 move_wall "alternate approach, give two paths, ro

{u'1': None,
 u'10': None,
 u'11': None,
 u'12': None,
 u'13': None,
 u'14': None,
 u'15': None,
 u'16': None,
 u'17': None,
 u'18': None,
 u'19': None,
 u'2': None,
 u'20': None,
 u'21': None,
 u'22': None,
 u'23': None,
 u'24': None,
 u'25': None,
 u'26': None,
 u'27': None,
 u'28': None,
 u'29': None,
 u'3': None,
 u'30': None,
 u'31': None,
 u'32': None,
 u'33': None,
 u'34': None,
 u'35': None,
 u'36': None,
 u'37': None,
 u'38': None,
 u'39': None,
 u'4': None,
 u'40': None,
 u'41': None,
 u'42': None,
 u'43': None,
 u'44': None,
 u'45': None,
 u'46': None,
 u'47': None,
 u'48': None,
 u'49': None,
 u'5': None,
 u'50': None,
 u'6': None,
 u'7': None,
 u'8': None,
 u'9': None}

In [23]:
#Print out all the codings for the square formation gesture, ordered by time
def print_events(participant):
    cond = adh.IdToCondition(str(participant["participant"]))
    if cond[1] != 1:
        print "\n{} - line formation".format(participant["participant"])
        #Only want events with a timestamp
        events = [e for e in participant["tasks"]["10"] if "time" in e.keys()]
        #Strip out examples
        events = [e for e in events if not e["example"]]
        events.sort(key=lambda e: e["time"])
        #Print them out
        for e in events:
            e_type = e["event_type"]
            e_desc = ""
            if "description" in e.keys():
                e_desc += " ".join(e["description"])
            if "command" in e.keys():
                e_desc += " ".join(e["command"])
            if "draw" in e.keys() and e["draw"] is not None:
                e_desc += " draw: " + " ".join(e["draw"])
            if "objects" in e.keys():
                e_desc += " ".join(e["objects"])
            
            
            print e_type, e_desc
throwaway = adh.apply(print_events)




48 - line formation
drag "single robot to top r quarter of screen"
tap "end of drag"

43 - line formation
other "threading select over all robots, diagonal drag up screen, then vertically downward to form line on r side of screen""robots"

49 - line formation
box_select "all robots"
pinch "One finger stays still on robots, other moves away to indicate line"

24 - line formation
lasso "all robots"
drag  draw: "vertical line"whitespace
drag "robots to previous line"
drag  draw: arrowhead"previous drag"
drag  draw: "vertical line"whitespace
drag  draw: "circle over robots"robots

25 - line formation
voice_command "get in a line"
drag  draw: "vertical line"whitespace

26 - line formation
tap robots
tap robots
tap whitespace

20 - line formation
drag "out from robots then into arc and above robots"

21 - line formation
drag  draw: "line across middle of screen"whitespace

23 - line formation
drag "robot, adjust position"
drag "robot, adjust position"
drag "robot"
drag "robot"
drag "robot"


In [24]:
#See what condition the single-moves square users were in
print "Square"
for user in [28,43,23,8,38,16]:
    print user, adh.IdToCondition(str(user))
    
print "Line"
#See what condition the single-moves line users were in
for user in [28,23,8,38]:
    print user, adh.IdToCondition(str(user))

Square
28 ('ten', 10)
43 ('ten', 10)
23 ('ten', 10)
8 ('ten', 10)
38 ('ten', 10)
16 ('unknown', 'X')
Line
28 ('ten', 10)
23 ('ten', 10)
8 ('ten', 10)
38 ('ten', 10)


In [25]:
#Figuring out the manipulation strategies
def print_crate(participant):
    cond = adh.IdToCondition(str(participant["participant"]))
    taskID = adh.taskMap["crate"][cond[1]]
    if taskID is not None: #crate_dispersed is not present in the 1-robot case
        print "\n{} - crate".format(participant["participant"])
        #Only want events with a timestamp
        events = [e for e in participant["tasks"][str(taskID)] if "time" in e.keys()]
        #Strip out examples
        events = [e for e in events if not e["example"]]
        events.sort(key=lambda e: e["time"])
        #Print them out
        for e in events:
            e_type = e["event_type"]
            e_desc = ""
            if "description" in e.keys():
                e_desc += " ".join(e["description"])
            if "command" in e.keys():
                e_desc += " ".join(e["command"])
            if "draw" in e.keys() and e["draw"] is not None:
                e_desc += " draw: " + " ".join(e["draw"])
            if "objects" in e.keys():
                e_desc += " ".join(e["objects"])
           
            print e_type, e_desc
    
throwaway = adh.apply(print_crate)


42 - crate
tap crate
tap "area A"

48 - crate
drag  draw: "square around crate"crate
drag "crate to A"
tap "end of path in A"

43 - crate
other "2f drag two robots to behind crate""two robots"
other "2f drag two robots to behind crate and through crate to A""two robots"

49 - crate
box_select "all robots"
tap "just under box"
other "hold one finger on group and draw crescent behind box""robot group/whitespace"
other "hold two fingers on group and draw crescent behind box with two fingers""robot group/whitespace"
tap "area A"

24 - crate
lasso "all robots"
drag "robots behind crate"
drag "crate to area A"

25 - crate
drag "robots to crate"
drag "just under crate to A"

26 - crate
drag "robots over crate"
drag "robots over crate"
drag "over crate to area A"

27 - crate
drag "robot behind crate"
drag "crate to A"

20 - crate
drag "crate to A"

21 - crate
drag "crate into A"

22 - crate
drag "robot behind crate and to area A"

23 - crate
drag "robot above crate"
drag "robot above crate"
d

In [26]:
#What condition were the drag-through users in?
drag_through_users = [1,6,12,30,37,36,22]
for user in drag_through_users:
    print user, adh.IdToCondition(str(user))

1 ('unknown', 'X')
6 ('unknown', 'X')
12 ('one', 1)
30 ('thousand', 1000)
37 ('one', 1)
36 ('unknown', 'X')
22 ('one', 1)


In [27]:
#Make a graph of all of the gestures used, as broken down by interface, position, and selection gestures
import json
with open("./all_participants_part_tagged.json", 'r') as infile:
    tagged_data = json.loads(infile.read())


In [28]:
interface_list = []
selection_list = []
position_list = []
for user in sorted(tagged_data.keys(), key=lambda x: int(x)):
    tasks = sorted(tagged_data[user]["tasks"].keys())
    for task in tasks:
        events = tagged_data[user]["tasks"][task]
        for event in events:
            if "interface" in event.keys() and event["interface"]:
                interface_list.append(event)
            if "position" in event.keys() and event["position"]:
                position_list.append(event)
            if "selection" in event.keys() and event["selection"]:
                selection_list.append(event)


In [29]:
def add_count(d, value):
    if value in d.keys():
        d[value] += 1
    else:
        d[value] = 1
        
interface_counts = {}
for event in interface_list:
    if event["event_type"] == "ui":
        if event["kind"] == "menu":
            add_count(interface_counts, "ui_menu")
        elif event["kind"] == "button":
            add_count(interface_counts, "ui_button")
        elif event["kind"] == "other":
            add_count(interface_counts, "ui_other")
    elif event["event_type"] == "drag":
        add_count(interface_counts, "drag")
    elif event["event_type"] == "voice_command":
        add_count(interface_counts, "voice")
    elif event["event_type"] == "pinch":
        if event["reverse"]:
            add_count(interface_counts, "rev_pinch")
        else:
            add_count(interface_counts, "pinch")
    elif event["event_type"] == "tap":
        if event["count"] == 1:
            add_count(interface_counts, "tap")
        elif event["count"] == 2:
            add_count(interface_counts, "double_tap")
        elif event["count"] == 3:
            add_count(interface_counts, "triple_tap")
        if event["hold"]:
            add_count(interface_counts, "hold")
    else:
        #Shouldn't happen
        print event["event_type"]
        

In [30]:
position_counts = {}

for event in position_list:
    if event["event_type"] == "drag":
        add_count(position_counts, "drag")
    elif event["event_type"] == "voice_command":
        add_count(position_counts, "voice")
    elif event["event_type"] == "pinch":
        if event["reverse"]:
            add_count(position_counts, "rev_pinch")
        else:
            add_count(position_counts, "pinch")
    elif event["event_type"] == "tap":
        if event["count"] == 1:
            add_count(position_counts, "tap")
        elif event["count"] == 2:
            add_count(position_counts, "double_tap")
        elif event["count"] == 3:
            add_count(position_counts, "triple_tap")
        if event["hold"]:
            add_count(position_counts, "hold")
    elif event["event_type"] == "other":
        add_count(position_counts, "other")
    elif event["event_type"] == "lasso":
        add_count(position_counts, "lasso")
    else:
        #Shouldn't happen
        print event["event_type"]


In [31]:
selection_counts = {}

for event in selection_list:
    if event["event_type"] == "lasso":
        add_count(selection_counts, "lasso")
    elif event["event_type"] == "box_select":
        add_count(selection_counts, "box_select")
    elif event["event_type"] == "drag":
        add_count(selection_counts, "drag")
    elif event["event_type"] == "pinch":
        if event["reverse"]:
            add_count(selection_counts, "rev_pinch")
        else:
            add_count(selection_counts, "pinch")
    elif event["event_type"] == "tap":
        if event["count"] == 1:
            add_count(selection_counts, "tap")
        elif event["count"] == 2:
            add_count(selection_counts, "double_tap")
        elif event["count"] == 3:
            add_count(selection_counts, "triple_tap")
        if event["hold"]:
            add_count(selection_counts, "hold")
    elif event["event_type"] == "other":
        add_count(selection_counts, "other")
    else:
        print event["event_type"]

In [32]:
#Convert to percentages
total = sum(selection_counts.values())
selection_counts_percent = {k:(float(v)/total) * 100 for k, v in selection_counts.items()}
total = sum(position_counts.values())
position_counts_percent = {k:(float(v)/total) * 100 for k, v in position_counts.items()}
total = sum(interface_counts.values())
interface_counts_percent = {k:(float(v)/total) * 100 for k, v in interface_counts.items()}

In [33]:
for item in sorted(position_counts_percent.items(), key=lambda x: -x[1]):
    print item[0],',',item[1]

drag , 75.0465549348
tap , 12.0577281192
other , 7.72811918063
rev_pinch , 1.8156424581
pinch , 1.16387337058
double_tap , 0.931098696462
hold , 0.837988826816
triple_tap , 0.372439478585
lasso , 0.0465549348231


In [121]:
#Count which tasks attracted the most UI operations
del ui_by_tasks
ui_by_tasks = {}
del ui_by_task_users
ui_by_task_users = {}


for user in adh.data.keys():
    for task in adh.data[user]["tasks"].keys():
        for event in adh.data[user]["tasks"][task]:
            if event["event_type"] == "ui":
                #Convert task id to a name
                taskName = adh.taskNumberToName(int(task), user)

                #Keep track of how many UI gestures were used
                if taskName in ui_by_tasks.keys():
                    ui_by_tasks[taskName] += 1
                else:
                    ui_by_tasks[taskName] = 1
                
                #Keep a list of how many users wanted UI for this task
                if taskName in ui_by_task_users.keys():
                    #print ".{}".format(user)
                    ui_by_task_users[taskName].add(user)
                    
                else:
                    #Ok, this is due to some ...special behavior... on the part of set.
                    #set(u'19') is creating a set from an iterable, so it gets you the set
                    #(u'1', u'9). Creating a set from a list like set([u'19']) actually puts
                    #the string into the set. Creating a set from a tuple with set((u'19'))
                    #throws away the tuple and iterates its innards.
                    #This means set(u'19') and set.add(u'19') do very different things.
                    ui_by_task_users[taskName] = set()
                    ui_by_task_users[taskName].add(user)
                    

In [127]:
#Get all the unique user ids who used the UI
all_ui_users = set()
for user_set in ui_by_task_users.values():
    for user in list(user_set):
        all_ui_users.add(user)
        
all_ui_users

{u'18',
 u'19',
 u'2',
 u'22',
 u'28',
 u'29',
 u'3',
 u'30',
 u'32',
 u'4',
 u'48',
 u'50',
 u'7'}

TypeError: unhashable type: 'set'